In [ ]:
from dataDownloader  import DataDownloader
from db_financialStatement import DB_FinancialStatement
from db_nyse import DB_NYSE
from portfolio import Portfolio
from assetAllocation import AssetAllocation


import datetime
import dataDownloader
import yfinance as yf;
import pandas as pd
import commonHelper


start_date = '2013-12-01'
end_date = '2025-08-13'

df_ncva_rank = DB_FinancialStatement.get_f_score_rank_table()
symbols = list(set(val for val in df_ncva_rank.values.ravel() if pd.notna(val)))
quarter_list = df_ncva_rank.columns.to_list()

date_dict = commonHelper.get_date_dict_by_quarter_lazy(quarter_list)
date_dict = commonHelper.get_trimmed_date_dict(date_dict, start_date, end_date)
date_dict = commonHelper.adjust_start_data_dict_by_quarter(date_dict, quarter_list[0])
oldest, latest = commonHelper.get_date_range_from_quarters(date_dict)

df = AssetAllocation.get_stock_data_with_ma(symbols=symbols, start_date=oldest, end_date=latest, mas=[10], type='ma_month', use_db_stock=True)
df = AssetAllocation.filter_close_last_month(df)
df = AssetAllocation.strategy_ncva(df, df_ncva_rank, date_dict)

display(df)


2022-10-22 00:00:00



1 Failed download:
['AZUL']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-10-22 00:00:00 -> 2025-08-13 00:00:00) (Yahoo error = "No data found, symbol may be delisted")')
